In [2]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import sqlite3


In [3]:
df = pd.read_csv('../data/Twitter.csv')

In [8]:
# Create a Database in AWS, with two diferents tables (tweets and users)

db = pymysql.connect(host = 'database-1.cxntqt2e7csj.us-east-1.rds.amazonaws.com',
                     user = 'admin',
                     password = '12345678',
                     cursorclass = pymysql.cursors.DictCursor
)

cursor = db.cursor()

In [9]:
# Create a new Database inside AWS
create_db = '''CREATE DATABASE Twitter'''
cursor.execute(create_db)

1

In [10]:
# Use the Database
cursor.connection.commit()
use_db = ''' USE Twitter'''
cursor.execute(use_db)

0

In [11]:
# Create the tweets table
create_table = '''
CREATE TABLE tweets (
    id_user INT,
    date DATETIME,
    tweets TEXT,
    primary key (id_user)
)
'''
cursor.execute(create_table)

0

In [12]:
# Create the dataset for getting inside of table
tweets = df[['Id del autor','Fecha del tweet','Tweets']]
tweets.columns = ['id_user','date','tweets']
tweets.head()

,id_user,date,tweets
0,1585726199485140993,2023-03-06 10:38:30+00:00,Las inscripciones se realizarán por Eventbrite...
1,1585726199485140993,2023-03-06 10:38:30+00:00,¡Por fin podemos publicar el cartel con los po...
2,41545659,2023-03-06 08:48:56+00:00,"Encuentro de la @mlopscommunity en Madrid: ""ML..."
3,925597948410580993,2023-03-04 22:32:56+00:00,@nodosenlared @snowded @TheBridge_Tech Muy bue...
4,276656109,2023-03-04 14:08:54+00:00,@PauMugarra @ToniDorta @TheBridge_Tech https:/...


In [13]:
# Create the usuarios table
create_table = '''
CREATE TABLE usuarios (
    id_user INT,
    username TEXT,
    primary key (id_user)
)
'''
cursor.execute(create_table)

0

In [14]:
# Create the dataset for getting inside of table
usuarios = df[['Id del autor','Nombre de usuario']]
usuarios.columns = ['id_user','username']
usuarios.head()

,id_user,username
0,1585726199485140993,ESOLIUPO
1,1585726199485140993,ESOLIUPO
2,41545659,nodosenlared
3,925597948410580993,RoccoLozano
4,276656109,joobid


In [15]:
# Get in all values we want 
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = 'admin', pw = '12345678', host = 'database-1.cxntqt2e7csj.us-east-1.rds.amazonaws.com', db = 'Twitter'))


In [16]:
# When you need to add new values, you shoud write down 'append', because in that case i will rewrite the values

tweets.to_sql(name='tweets', con=engine, if_exists= 'replace', index=False)

128

In [17]:
# When you need to add new values, you shoud write down 'append', because in that case i will rewrite the values

usuarios.to_sql(name='usuarios', con=engine, if_exists= 'replace', index=False)

128

In [18]:
db.commit()

In [19]:
sql = '''SELECT * FROM usuarios'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'id_user': 1585726199485140993, 'username': 'ESOLIUPO'},
 {'id_user': 1585726199485140993, 'username': 'ESOLIUPO'},
 {'id_user': 41545659, 'username': 'nodosenlared'},
 {'id_user': 925597948410580993, 'username': 'RoccoLozano'},
 {'id_user': 276656109, 'username': 'joobid'},
 {'id_user': 1003872445, 'username': 'PauMugarra'},
 {'id_user': 763727449, 'username': 'OmarjLand'},
 {'id_user': 1207228796593745920, 'username': 'decentralized_b'},
 {'id_user': 1207228796593745920, 'username': 'decentralized_b'},
 {'id_user': 2603024598, 'username': 'rperezmarco'},
 {'id_user': 1207228796593745920, 'username': 'decentralized_b'},
 {'id_user': 1099588676181471233, 'username': 'turoninos'},
 {'id_user': 1685197165, 'username': 'Metricool_es'},
 {'id_user': 1685197165, 'username': 'Metricool_es'},
 {'id_user': 1288769305014939650, 'username': '42UrdulizFTef'},
 {'id_user': 1003872445, 'username': 'PauMugarra'},
 {'id_user': 1099588676181471233, 'username': 'turoninos'},
 {'id_user': 153019474378

In [20]:
sql = '''SELECT * FROM tweets'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'id_user': 1585726199485140993,
  'date': '2023-03-06 10:38:30+00:00',
  'tweets': 'Las inscripciones se realizarán por Eventbrite. Cualquier duda estamos por MD o al correo info@esoliupo.es\nhttps://t.co/xhVr7VXcF3\n\n@epsUPO @TheBridge_Tech'},
 {'id_user': 1585726199485140993,
  'date': '2023-03-06 10:38:30+00:00',
  'tweets': '¡Por fin podemos publicar el cartel con los ponentes para el #cybersecTalks! Si estás interesadx en alguna de las temáticas que aparecen no dudes en venir, es totalmente gratuito gracias al patrocinio de @TheBridge_Tech y la colaboración de @epsUPO @pablodeolavide \n(+ info abajo) https://t.co/JF85MDGmXc'},
 {'id_user': 41545659,
  'date': '2023-03-06 08:48:56+00:00',
  'tweets': 'Encuentro de la @mlopscommunity en Madrid: "MLOPs: desde las trincheras" https://t.co/CHojEAs7ij en @TheBridge_Tech Ven a conocer la práctica #MLOps en Correcto., @Busuu @wallapop https://t.co/odie5CAuHU'},
 {'id_user': 925597948410580993,
  'date': '2023-03-04 22:32:56+00:00',
  '

In [54]:
# Now, we want to select all values with splitted month and year, count the number of tweets and join with the other table and user that post more than 5 times
sql = '''SELECT 
    extract(month from date) as month, 
    extract(year from date) as year,
    COUNT(u.username) as tweet_count,
    username
    FROM tweets as t
    INNER JOIN usuarios as u on t.id_user = u.id_user
    GROUP BY month
    HAVING tweet_count > 5'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'month': 3, 'year': 2023, 'tweet_count': 27, 'username': 'ESOLIUPO'},
 {'month': 1, 'year': 2023, 'tweet_count': 189, 'username': 'nodosenlared'},
 {'month': 2, 'year': 2023, 'tweet_count': 46, 'username': 'PauMugarra'},
 {'month': 12,
  'year': 2022,
  'tweet_count': 160,
  'username': 'BitcoinTuesday_'},
 {'month': 11, 'year': 2022, 'tweet_count': 414, 'username': 'jorgegrev'}]

In [55]:
pd.DataFrame(mi_tabla)

,month,year,tweet_count,username
0,3,2023,27,ESOLIUPO
1,1,2023,189,nodosenlared
2,2,2023,46,PauMugarra
3,12,2022,160,BitcoinTuesday_
4,11,2022,414,jorgegrev


In [7]:
# Delete BBDD
#drop_db = '''DROP DATABASE Twitter'''
#cursor.execute(drop_db)

2

* Create a BBDD in SQLite

In [28]:
connection = sqlite3.connect("Twitter.db")
crsr = connection.cursor()

In [29]:
query_tweets = '''
CREATE TABLE tweets (
    id_user INT,
    date DATETIME,
    tweets TEXT,
    primary key (id_user)
)
'''
crsr.execute(query_tweets)

In [30]:
query_usuarios = '''
CREATE TABLE usuarios (
    id_user INT,
    username TEXT,
    primary key (id_user)
)
'''
crsr.execute(query_usuarios)

In [31]:
engine_sqlite = create_engine('sqlite:///Twitter.db')

In [32]:
tweets.to_sql(name='tweets', con=engine_sqlite, if_exists= 'replace', index=False)

128

In [33]:
usuarios.to_sql(name='usuarios', con=engine_sqlite, if_exists= 'replace', index=False)

128

In [34]:
connection.commit()

In [35]:
connection.close()